In [179]:
# simulating amm in with different fee amounts
from functools import total_ordering
import pandas as pd

In [180]:
# Load AAPL and MSFT data
aapl_data = pd.read_excel("aapl_10-25-2024_1_min.xlsx")
msft_data = pd.read_excel("msft_25-10-2024_1_min.xlsx")
aapl_data['Time'] = pd.to_datetime(aapl_data['Time'], format="%m/%d/%Y %I:%M:%S %p")
msft_data['Time'] = pd.to_datetime(msft_data['Time'], format="%m/%d/%Y %I:%M:%S %p")

print(aapl_data)

       Open    High     Low   Close  Volume                Time
0    227.75  227.95  227.46  227.56    3426 2024-10-25 00:00:00
1    227.50  228.24  227.34  227.89    5158 2024-10-25 00:01:00
2    227.80  228.02  227.53  227.86    5954 2024-10-25 00:02:00
3    227.87  227.88  227.88  227.89     877 2024-10-25 00:03:00
4    227.89  228.20  227.80  228.20    1003 2024-10-25 00:04:00
..      ...     ...     ...     ...     ...                 ...
897  230.90  230.90  230.82  230.85      46 2024-10-25 15:55:00
898  230.80  230.80  230.80  230.80     210 2024-10-25 15:56:00
899  230.85  230.85  230.85  230.85     202 2024-10-25 15:57:00
900  230.92  230.99  230.92  230.97      60 2024-10-25 15:58:00
901  231.00  231.00  231.00  231.00      44 2024-10-25 15:59:00

[902 rows x 6 columns]


In [181]:
# Merge unique timestamps from both datasets and sort them
unique_times = pd.concat([aapl_data['Time'], msft_data['Time']]).drop_duplicates().sort_values()
print(unique_times)

0     2024-10-25 00:00:00
1     2024-10-25 00:01:00
2     2024-10-25 00:02:00
3     2024-10-25 00:03:00
4     2024-10-25 00:04:00
              ...        
897   2024-10-25 15:55:00
898   2024-10-25 15:56:00
899   2024-10-25 15:57:00
900   2024-10-25 15:58:00
901   2024-10-25 15:59:00
Name: Time, Length: 931, dtype: datetime64[ns]


In [182]:
# Define manual start and end indices (default 0 for full range)
manual_start_index = 10  # Set to desired index, e.g., 4
manual_end_index = len(unique_times) - 1  # Set to desired index, e.g., 10

# Validate and adjust the indices if out of range
if manual_start_index < 0 or manual_start_index >= len(unique_times):
    manual_start_index = 0
if manual_end_index < 0 or manual_end_index >= len(unique_times):
    manual_end_index = len(unique_times) - 1

# Get the corresponding times for the selected indices
period_start = unique_times.iloc[manual_start_index]
period_end = unique_times.iloc[manual_end_index]

# Filter unique times within the selected range
filtered_times = unique_times.iloc[manual_start_index:manual_end_index + 1]

# Print the selected period and filtered times for verification
print(f"Simulation Period: {period_start} to {period_end}")
print(filtered_times)

Simulation Period: 2024-10-25 00:10:00 to 2024-10-25 15:59:00
9     2024-10-25 00:10:00
10    2024-10-25 00:11:00
11    2024-10-25 00:12:00
11    2024-10-25 00:13:00
12    2024-10-25 00:14:00
              ...        
897   2024-10-25 15:55:00
898   2024-10-25 15:56:00
899   2024-10-25 15:57:00
900   2024-10-25 15:58:00
901   2024-10-25 15:59:00
Name: Time, Length: 921, dtype: datetime64[ns]


In [183]:
# Define assets and initialize parameters
#assets = ["USD", "AAPL", "MSFT"]
#opening_spots =  [1, 227.75, 426.38]
data = {"AAPL": aapl_data, "MSFT": msft_data}
#spots = {"USD": opening_spots[0], "AAPL": opening_spots[1], "MSFT": opening_spots[2]}
#weights = {"USD": 0.4, "AAPL": 0.3, "MSFT": 0.3}
initial_pool = 250000
#balances = {asset: (initial_pool * weights[asset]) / spots[asset] for asset in assets}
starting_table = pd.DataFrame({
    "asset": ["USD", "AAPL", "MSFT"],
    "spot": [1, 227.75, 426.38],
    "weight": [0.4, 0.3, 0.3],
})

starting_table["value"] = initial_pool * starting_table["weight"]
starting_table["balance"] = starting_table["value"] / starting_table["spot"]

starting_table.set_index("asset", inplace=True)

starting_table

,spot,weight,value,balance
asset,,,,
USD,1.00,0.4,100000.0,100000.000000
AAPL,227.75,0.3,75000.0,329.308452
MSFT,426.38,0.3,75000.0,175.899432


In [184]:
# Fee values for different scenarios
#fee_values = [0.1, 0.25, 0.3, 0.4, 0.5, 0.75, 0.9, 1, 1.25, 1.5, 2, 2.5]
fee_values = [1, 2, 3]
order_sizes = [0.25, 1, 2.5]

base_asset = starting_table.index[0]
if True:
  print(f"Base asset: {base_asset}")

  for asset in starting_table.index:
    print(asset)

Base asset: USD
USD
AAPL
MSFT


In [185]:
#Calculate invariant based on balances and weights.
def calculate_invariant(inv_table):
    #inv = 1
    #for asset in assets:
    #    inv *= balances[asset] ** weights[asset]
    #for asset in assets_table.items():
        #inv *= asset["balance"] ** asset["weight"]
    #return inv
    return (inv_table["balance"] ** inv_table["weight"]).prod()

In [186]:
def print_pool_standing(info_table):
    #pd.DataFrame(info_table)
    print(f"Assets: {info_table.index}")
    #print(f"Weights: {weights}")
    print(f"Balances: {info_table["balance"]}")
    print(f"Inv: {calculate_invariant(info_table)}")
    #print("Values: ")
    total_pool_value = 0
    #for asset in assets:
        #total_pool_value += balances[asset] * opens[asset]
        #print("Asset ", asset, " value: ", balances[asset] * opens[asset])
        #inv *= balances[asset] ** weights[asset])
    print(f"Total pool value: {info_table["value"].sum()}")

In [187]:
def pretty_print_table(df):
    formatted_df = df.copy()  # Create a copy to avoid modifying the original DataFrame
    for col in formatted_df.select_dtypes(include=['float', 'int']).columns:
        formatted_df[col] = formatted_df[col].apply(lambda x: f"{x:,.4f}")
    print(formatted_df)

In [188]:
results = []
result_tables = []
direction = {}
#sim_table = starting_table
for order_size in order_sizes:
    print(f"Order Size simulation: {order_size}")
    # Loop through each fee variation
    for fee_percentage in fee_values:
        print(f"Fee % simulation: {fee_percentage}")
        sim_table = starting_table.copy()
        trades = 0
        fees_earned = 0
        fee = fee_percentage / 100
        # Initialize balances and invariant
        #balances = {asset: (initial_pool * weights[asset]) / spots[asset] for asset in assets}


        trades_log = []
        end_of_period_values = []

        # Loop through each unique time period
        # Fetch data for each asset at the current time and update last close prices if needed
        open_prices = {}
        last_close_prices = {}
        #open_prices[asset] = spots["USD"]
        #last_close_prices[asset] = spots["USD"]
        times = 0
        missing_data_log = {}
        for time in filtered_times:
            '''times += 1
            if times > 2:
                break'''
            #if trades > 10:
                #rint("breaking")
                #break

            if False:
                print(time)
                print(sim_table)
            missing_data_assets = set()
            #
            for asset in sim_table.index:
                if asset == base_asset:
                    #open_prices[asset] = spots["USD"]
                    #last_close_prices[asset] = spots["USD"]
                    pass
                else:
                    direction[asset] = None
                    asset_data = data[asset]
                    row = asset_data[asset_data['Time'] == time]
                    if row.empty:
                        print(f"No data for asset {asset} at time {time}")
                        missing_data_assets.add(asset)  # Flag the asse
                        # Log the missing data for later analysis
                        if time not in missing_data_log:
                            missing_data_log[time] = []
                        missing_data_log[time].append(asset)
                        continue
                    else:
                        sim_table.at[asset, "spot"] = row['Close'].values[0]
                    '''if not row.empty:
                        sim_table.at[asset, "spot"] = row['Open'].values[0]
                        last_close_prices[asset] = row['Close'].values[0]
                    else:
                        open_prices[asset] = sim_table.at[asset, "spot"]'''
                        

            # Trading loop for each asset
            continue_trading = True
            while continue_trading:
                continue_trading = False

                for asset in sim_table.index:
                    if asset == base_asset or asset in missing_data_assets:  # Skip flagged assets for this time
                        continue

                    #print(asset)

                    # Fetch high and low prices for the asset
                    row = data[asset][data[asset]['Time'] == time]
                    if not row.empty:
                        high_price = row['High'].values[0]
                        low_price = row['Low'].values[0]

                        # Calculate bid and ask prices for the current asset
                        '''other_balances_product = 1
                        for other_asset in assets:
                            if other_asset != asset and other_asset != "USD":
                                #print("other asset: ", other_asset, " balance: ", balances[other_asset], " weight: ", weights[other_asset])
                                other_balances_product *= balances[other_asset] ** weights[other_asset]

                        inv = calculate_invariant(balances, weights)'''
                        #inv = calculate_invariant(sim_table)
                        balance = sim_table.at[asset, "balance"]
                        weight = sim_table.at[asset, "weight"]

                        moe_balance = sim_table.at[base_asset, "balance"]
                        moe_weight = sim_table.at[base_asset, "weight"]
                        
                        if False:
                            print(f"Balance: {balance}, weight: {weight}")
                            other_balances_product = inv / (balance ** weight)
                            print(f"Inv: {inv}, Other bal product: {other_balances_product}")
                        #print("other asset balances produt: ", other_balances_product)
                        #print("balance of the asset: ", balances[asset])
                        #print("invariant: ", balances[asset]**weights[asset] * other_balances_product)

                        #bid_amount_wo_fee = abs(((inv / ((balance + order_size) ** weight * other_balances_product)) ** (1 / sim_table.at["USD", "weight"]) - sim_table.at["USD", "balance"]))
                        #ask_amount_wo_fee = abs(((inv / ((balance - order_size) ** weight * other_balances_product)) ** (1 / sim_table.at["USD", "weight"]) - sim_table.at["USD", "balance"]))

                        #bid_amount_wo_fee = sim_table.at["USD", "balance"] - (((sim_table.at["USD", "balance"]**sim_table.at["USD", "weight"] * balance**weight) / ((balance + order_size)**weight)))**(1 / sim_table.at["USD", "weight"])
                        #ask_amount_wo_fee = -sim_table.at["USD", "balance"] + (((sim_table.at["USD", "balance"]**sim_table.at["USD", "weight"] * balance**weight) / ((balance - order_size)**weight)))**(1 / sim_table.at["USD", "weight"])
                        
                        bid_amount_wo_fee = moe_balance - (((moe_balance**moe_weight * balance**weight) / ((balance + order_size)**weight)))**(1 / moe_weight)
                        ask_amount_wo_fee = -moe_balance+ (((moe_balance**moe_weight * balance**weight) / ((balance - order_size)**weight)))**(1 / moe_weight)
                        
                        bid_amount = bid_amount_wo_fee * (1 - fee)
                        ask_amount = ask_amount_wo_fee * (1 + fee)

                        bid_price = bid_amount / order_size
                        ask_price = ask_amount / order_size

                        if False:
                            print(asset, " bid: ", bid_price)
                            print(asset, " ask: ", ask_price)

                        # Sell asset
                        if high_price >= ask_price and direction[asset] != "buy":
                            if balance >= order_size:
                                direction[asset] = "sell"
                                trades += 1
                                if True:
                                    print("Trade: ", trades)
                                    print_pool_standing(sim_table)
                                    #trades_log.append({'Time': time, 'Asset': asset, 'Action': 'Sell', 'Shares': 1, 'Price': ask_price})
                                    #print({'Time': time, 'Asset': asset, 'Action': 'Sell', 'Shares': 1, 'Price': ask_price})
                                    trade_printout = {'Time': time, 'Asset': asset, 'Action': 'Sell', 'Quantity': order_size, 'Amount': round(float(ask_amount), 4), 'Price': round(float(ask_amount_wo_fee), 4)}
                                    trades_log.append(trade_printout)
                                    print(trade_printout)

                                sim_table.at[base_asset, "balance"] += ask_amount
                                sim_table.at[asset, "balance"] -= order_size
                                fees_earned += ask_amount - ask_amount_wo_fee

                                #inv = calculate_invariant(sim_table)
                                if True:
                                    print(f"Fee earned: {round(ask_amount - ask_amount_wo_fee, 4)} (total: {round(fees_earned, 4)}), With fee: {round(ask_amount, 4)}, without fee {round(ask_amount_wo_fee, 4)}")
                                    print_pool_standing(sim_table)
                                    print()
                                continue_trading = True  # Check for more trades within the period
                                break

                        # Buy asset
                        elif low_price <= bid_price and direction[asset] != "sell":
                            if moe_balance >= bid_price:
                                direction[asset] = "buy"

                                if True:
                                    trades += 1
                                    print("Trade: ", trades)
                                    print_pool_standing(sim_table)
                                    trade_printout = {'Time': time, 'Asset': asset, 'Action': 'Buy', 'Quantity': order_size, 'Amount': round(float(bid_amount), 4), 'Price': round(float(bid_price), 4)}
                                    trades_log.append(trade_printout)
                                    print(trade_printout)
                                sim_table.at[base_asset, "balance"] -= bid_amount
                                sim_table.at[asset, "balance"] += order_size
                                fees_earned += bid_amount_wo_fee - bid_amount

                                #inv = calculate_invariant(balances, weights)
                                if True:
                                    print(f"Fee earned: {round(bid_amount_wo_fee - bid_amount, 4)} (total: {round(fees_earned, 4)}), With fee: {round(bid_amount, 4)}, without fee {round(bid_amount_wo_fee, 4)}")
                                    print_pool_standing(sim_table)
                                    print()
                                continue_trading = True  # Check for more trades within the period
                                break

            for asset in sim_table.index:
                if asset == base_asset:
                    pass
                else:
                    row = asset_data[asset_data['Time'] == time]
                    # sim_table.at[asset, "spot"] = row['Close'].values[0]

                    sim_table.at[asset, "value"] = sim_table.at[asset, "spot"] * sim_table.at[asset, "balance"]

            # Calculate end-of-period pool value
            '''pool_value = balances["USD"]
            il_formula_end_value = 1
            for asset in assets:
                if asset != "USD":
                    pool_value += balances[asset] * last_close_prices[asset]
                    il_formula_end_value *= (last_close_prices[asset] / )
            end_of_period_values.append(pool_value)'''
            end_of_period_values.append(sim_table["value"].sum())

        # Record the final pool value for the current fee
        final_pool_value = end_of_period_values[-1]
        results.append({'Order Size': order_size, 'Fee %': fee_percentage, 'Final_Pool_Value': final_pool_value, "Fees": fees_earned, "Trades": trades})
        result_tables.append(sim_table)

# Convert results to DataFrame for comparison
results_df = pd.DataFrame(results)


Order Size simulation: 0.25
Fee % simulation: 1
No data for asset MSFT at time 2024-10-25 00:07:00
No data for asset AAPL at time 2024-10-25 00:10:00
No data for asset AAPL at time 2024-10-25 00:12:00
No data for asset MSFT at time 2024-10-25 00:13:00
No data for asset AAPL at time 2024-10-25 00:15:00
No data for asset MSFT at time 2024-10-25 00:21:00
No data for asset MSFT at time 2024-10-25 00:24:00
No data for asset MSFT at time 2024-10-25 00:27:00
No data for asset MSFT at time 2024-10-25 00:31:00
No data for asset MSFT at time 2024-10-25 00:33:00
No data for asset MSFT at time 2024-10-25 00:35:00
No data for asset MSFT at time 2024-10-25 00:36:00
No data for asset AAPL at time 2024-10-25 00:39:00
No data for asset MSFT at time 2024-10-25 00:41:00
No data for asset MSFT at time 2024-10-25 00:42:00
No data for asset AAPL at time 2024-10-25 00:43:00
No data for asset MSFT at time 2024-10-25 00:47:00
No data for asset MSFT at time 2024-10-25 00:48:00
No data for asset MSFT at time 202

In [189]:
print(f"Base asset: {base_asset}")

print("Starting table (V0):")
pretty_print_table(starting_table)
print("   Value:               ", f"{starting_table['value'].sum():,.4f}")
print()

#print(sim_table)

print("Hold table (VH):")
hold_table = starting_table.copy()
hold_table['spot'] = sim_table['spot']
hold_table['value'] = hold_table['spot'] * hold_table['balance']
pretty_print_table(hold_table)
print("   Value:               ", f"{hold_table['value'].sum():,.4f}")
print()

print("Summary: ")
#print(results_df)
pretty_print_table(results_df)

import json

if True:
  print()
  print()
  print("End tables:")
  table_index = 1
  for result, table in zip(results, result_tables):
      # Print the summary for the table
      print(f"    Table: {table_index}")
      table_index += 1
      print(json.dumps(result, indent=4))
      # Print the table
      pretty_print_table(table)
      print("   Value:               ", f"{table['value'].sum():,.4f}")

Base asset: USD
Starting table (V0):
           spot  weight         value       balance
asset                                              
USD      1.0000  0.4000  100,000.0000  100,000.0000
AAPL   227.7500  0.3000   75,000.0000      329.3085
MSFT   426.3800  0.3000   75,000.0000      175.8994
   Value:                250,000.0000

Hold table (VH):
           spot  weight         value       balance
asset                                              
USD      1.0000  0.4000  100,000.0000  100,000.0000
AAPL   231.0000  0.3000   76,070.2525      329.3085
MSFT   427.4871  0.3000   75,194.7383      175.8994
   Value:                251,264.9907

Summary: 
  Order Size   Fee % Final_Pool_Value     Fees   Trades
0     0.2500  1.0000     250,915.8432  68.5331  74.0000
1     0.2500  2.0000     251,773.4649  48.8153  26.0000
2     0.2500  3.0000     251,906.2214  19.0466   6.0000
3     1.0000  1.0000     250,964.9649  70.1034  19.0000
4     1.0000  2.0000     251,657.9649  42.7305   6.0000
5 